<a href="https://colab.research.google.com/github/bradymiller2310/FantasyFootballDashboard/blob/main/ESPN_FF_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install espn-api
!pip install pandas
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully uninstalled urllib3-2.3.0


In [8]:
from espn_api.football import League

In [9]:
league = League(league_id=1292514, year=2024, espn_s2="AECPNWyLvfnlcFfE0Xb73O%2BP5ZU9n1G%2FSnP3ixxvuqSwQRcQe7bEQfBuz3YBNeRluTYofdhX2aTajj0jZSF%2B8i9ewdk8Dwf6p9G215F2FjmI06wPKyhjleL4WzaT3PCiP15FDXe55YNMkQeJXgbsCuK1VHbdQ1Nxidq0JxAHezRn2zAt6WwNtSqrF0k2nNHV33VpJqPmMk4msFmaoCaz0UAp95xxy1YmXyEUnZWgA60HJkbsgYkZ8AaESwFENsPYdUvKYqCwnZLjvgI5varhOTDYekjvokdtaCoxJfzOdGLJ3g%3D%3D", swid="{D68FE5F2-46CE-421C-B248-DDA01360BB34}")
print(league.teams)  #Teams in league

[Team(Fightin' Furries), Team(Quon Solo), Team(Jen-eral ⚔️), Team(Captain Sweatpants), Team(Salmon LipBalm !!), Team(Graham’s Groupie), Team(pop-pop's bible study), Team(Bucktown Bandits), Team(bird gang), Team(bungalicious  💅)]


In [ ]:
for week in range(1, 15):
    print(f"Week {week} Schedule:")
    for matchup in league.scoreboard(week=week):
        print(f"{matchup.home_team} vs {matchup.away_team}")

Week 1 Schedule:
Team(Bucktown Bandits) vs Team(Graham’s Groupie)
Team(bungalicious  💅) vs Team(Fightin' Furries)
Team(bird gang) vs Team(Salmon LipBalm !!)
Team(pop-pop's bible study) vs Team(Quon Solo)
Team(Jen-eral ⚔️) vs Team(Captain Sweatpants)
Week 2 Schedule:
Team(Fightin' Furries) vs Team(Salmon LipBalm !!)
Team(Quon Solo) vs Team(Bucktown Bandits)
Team(Captain Sweatpants) vs Team(Graham’s Groupie)
Team(pop-pop's bible study) vs Team(bungalicious  💅)
Team(Jen-eral ⚔️) vs Team(bird gang)
Week 3 Schedule:
Team(Quon Solo) vs Team(Captain Sweatpants)
Team(Fightin' Furries) vs Team(pop-pop's bible study)
Team(Salmon LipBalm !!) vs Team(Jen-eral ⚔️)
Team(Bucktown Bandits) vs Team(bungalicious  💅)
Team(Graham’s Groupie) vs Team(bird gang)
Week 4 Schedule:
Team(pop-pop's bible study) vs Team(Jen-eral ⚔️)
Team(bungalicious  💅) vs Team(Quon Solo)
Team(bird gang) vs Team(Captain Sweatpants)
Team(Bucktown Bandits) vs Team(Fightin' Furries)
Team(Graham’s Groupie) vs Team(Salmon LipBalm !!)


In [ ]:
free_agents = league.free_agents(size=20, position='QB')  # Get top 20 available QBs

print("Available Free Agents (QB):")
for player in free_agents:
    print(f"  - {player.name} ({player.proTeam})")

Available Free Agents (QB):
  - C.J. Stroud (HOU)
  - Tua Tagovailoa (MIA)
  - Anthony Richardson (IND)
  - Aaron Rodgers (NYJ)
  - Kirk Cousins (ATL)
  - Bryce Young (CAR)
  - Michael Penix Jr. (ATL)
  - Trevor Lawrence (JAX)
  - Jameis Winston (CLE)
  - Derek Carr (NO)
  - Justin Fields (PIT)
  - Joe Flacco (IND)
  - Deshaun Watson (CLE)
  - Cooper Rush (DAL)
  - Daniel Jones (MIN)
  - Tom Brady (TB)
  - Mason Rudolph (TEN)
  - Mac Jones (JAX)
  - Tommy DeVito (NYG)
  - Aidan O'Connell (LV)


In [ ]:
for team in league.teams:
    print(f"\nTeam: {team.team_name}")
    print("Roster:")
    for player in team.roster:
        print(f"  - {player.name} ({player.position})")


Team: Fightin' Furries
Roster:
  - Tyreek Hill (WR)
  - Isiah Pacheco (RB)
  - Joe Mixon (RB)
  - Patrick Mahomes (QB)
  - Dalton Kincaid (TE)
  - Zay Flowers (WR)
  - Jaxon Smith-Njigba (WR)
  - Gus Edwards (RB)
  - Matthew Stafford (QB)
  - Kareem Hunt (RB)
  - Broncos D/ST (D/ST)
  - David Njoku (TE)
  - Caleb Williams (QB)
  - DeAndre Hopkins (WR)
  - Bills D/ST (D/ST)
  - Matthew Wright (K)

Team: Quon Solo
Roster:
  - Justin Jefferson (WR)
  - Kyren Williams (RB)
  - Marvin Harrison Jr. (WR)
  - Mike Evans (WR)
  - D'Andre Swift (RB)
  - Brian Robinson Jr. (RB)
  - Kyler Murray (QB)
  - J.K. Dobbins (RB)
  - Geno Smith (QB)
  - Chris Boswell (K)
  - Romeo Doubs (WR)
  - Mark Andrews (TE)
  - Bears D/ST (D/ST)
  - Courtland Sutton (WR)
  - Josh Downs (WR)
  - Evan Engram (TE)

Team: Jen-eral ⚔️
Roster:
  - Derrick Henry (RB)
  - A.J. Brown (WR)
  - Alvin Kamara (RB)
  - Jalen Hurts (QB)
  - Calvin Ridley (WR)
  - Raheem Mostert (RB)
  - Jets D/ST (D/ST)
  - Pat Freiermuth (TE)
  

In [ ]:
week_number = 1
box_scores = league.box_scores(week=week_number)

for matchup in box_scores:
    print(f"\nMatchup: {matchup.home_team} vs {matchup.away_team}")
    print(f"  Home Score: {matchup.home_score}")
    print(f"  Away Score: {matchup.away_score}")


Matchup: Team(Bucktown Bandits) vs Team(Graham’s Groupie)
  Home Score: 73.0
  Away Score: 96.0

Matchup: Team(bungalicious  💅) vs Team(Fightin' Furries)
  Home Score: 101.0
  Away Score: 94.0

Matchup: Team(bird gang) vs Team(Salmon LipBalm !!)
  Home Score: 59.0
  Away Score: 101.0

Matchup: Team(pop-pop's bible study) vs Team(Quon Solo)
  Home Score: 79.0
  Away Score: 97.0

Matchup: Team(Jen-eral ⚔️) vs Team(Captain Sweatpants)
  Home Score: 82.0
  Away Score: 110.0


In [ ]:
# getting injury status of free agents (not on rosters)

free_agents = league.free_agents()
i=0
for player in free_agents:
  player_obj = free_agents[i]
  print(f"Player: {player_obj.name}, Injury Status: {player_obj.injuryStatus}, Injured: {player_obj.injured}")
  i+=1

Player: Ladd McConkey, Injury Status: ACTIVE, Injured: False
Player: Keenan Allen, Injury Status: ACTIVE, Injured: False
Player: Michael Pittman Jr., Injury Status: QUESTIONABLE, Injured: False
Player: Steelers D/ST, Injury Status: [], Injured: False
Player: C.J. Stroud, Injury Status: ACTIVE, Injured: False
Player: Kyle Pitts, Injury Status: ACTIVE, Injured: False
Player: Adam Thielen, Injury Status: ACTIVE, Injured: False
Player: Chris Olave, Injury Status: QUESTIONABLE, Injured: False
Player: Tucker Kraft, Injury Status: ACTIVE, Injured: False
Player: 49ers D/ST, Injury Status: [], Injured: False
Player: Jerome Ford, Injury Status: QUESTIONABLE, Injured: False
Player: Texans D/ST, Injury Status: [], Injured: False
Player: Quentin Johnston, Injury Status: ACTIVE, Injured: False
Player: Tua Tagovailoa, Injury Status: QUESTIONABLE, Injured: False
Player: Nick Chubb, Injury Status: QUESTIONABLE, Injured: False
Player: Anthony Richardson, Injury Status: QUESTIONABLE, Injured: False
Playe

In [ ]:
# getting injury status of players on rosters
for team in league.teams:
    print(f"\nTeam: {team.team_name}")
    for player in team.roster:
        print(f"Player: {player.name}, Injury Status: {player.injuryStatus}, Injured: {player.injured}")


Team: Fightin' Furries
Player: Tyreek Hill, Injury Status: QUESTIONABLE, Injured: False
Player: Isiah Pacheco, Injury Status: ACTIVE, Injured: False
Player: Joe Mixon, Injury Status: ACTIVE, Injured: False
Player: Patrick Mahomes, Injury Status: ACTIVE, Injured: False
Player: Dalton Kincaid, Injury Status: ACTIVE, Injured: False
Player: Zay Flowers, Injury Status: QUESTIONABLE, Injured: False
Player: Jaxon Smith-Njigba, Injury Status: ACTIVE, Injured: False
Player: Gus Edwards, Injury Status: ACTIVE, Injured: False
Player: Matthew Stafford, Injury Status: ACTIVE, Injured: False
Player: Kareem Hunt, Injury Status: ACTIVE, Injured: False
Player: Broncos D/ST, Injury Status: NORMAL, Injured: False
Player: David Njoku, Injury Status: QUESTIONABLE, Injured: False
Player: Caleb Williams, Injury Status: QUESTIONABLE, Injured: False
Player: DeAndre Hopkins, Injury Status: ACTIVE, Injured: False
Player: Bills D/ST, Injury Status: NORMAL, Injured: False
Player: Matthew Wright, Injury Status: AC

In [ ]:
# getting starting roster for each week

for team in league.teams:
    print(f"\nTeam: {team.team_name} (Week {week})")

    # Get the team's roster for the given week
    for player in team.roster:
        if player.lineupSlot != "BE" and player.lineupSlot != "IR":  # Exclude Bench and IR players
            print(f"  - {player.name} ({player.lineupSlot})")



Team: Fightin' Furries (Week 14)
  - Tyreek Hill (WR)
  - Isiah Pacheco (RB)
  - Patrick Mahomes (QB)
  - Jaxon Smith-Njigba (RB/WR/TE)
  - Kareem Hunt (RB)
  - David Njoku (TE)
  - DeAndre Hopkins (WR)
  - Bills D/ST (D/ST)
  - Matthew Wright (K)

Team: Quon Solo (Week 14)
  - Justin Jefferson (WR)
  - Kyren Williams (RB)
  - D'Andre Swift (RB)
  - Brian Robinson Jr. (RB/WR/TE)
  - Kyler Murray (QB)
  - Chris Boswell (K)
  - Mark Andrews (TE)
  - Bears D/ST (D/ST)
  - Courtland Sutton (WR)

Team: Jen-eral ⚔️ (Week 14)
  - Derrick Henry (RB)
  - A.J. Brown (WR)
  - Alvin Kamara (RB)
  - Jalen Hurts (QB)
  - Calvin Ridley (WR)
  - Jets D/ST (D/ST)
  - Cameron Dicker (K)
  - Tank Bigsby (RB/WR/TE)
  - Jonnu Smith (TE)

Team: Captain Sweatpants (Week 14)
  - CeeDee Lamb (WR)
  - Garrett Wilson (WR)
  - James Conner (RB)
  - Jake Elliott (K)
  - Mike Gesicki (TE)
  - Rico Dowdle (RB)
  - Will Levis (QB)
  - Chargers D/ST (D/ST)
  - Khalil Shakir (RB/WR/TE)

Team: Salmon LipBalm !! (Week 1

### **Getting statistical data from xlsx files**

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import pandas as pd

file_path = "/content/drive/My Drive/ff_data.xlsx"

# Load specific sheets
season_stats_all = pd.read_excel(file_path, sheet_name="Weekly", engine="openpyxl")
weekly_stats_all = pd.read_excel(file_path, sheet_name="Season", engine="openpyxl")

# Display first few rows
print(season_stats_all.head())
print(weekly_stats_all.head())

TypeError: expected <class 'int'>

In [16]:
# Define the range of weeks
weeks = range(1, 19)  # Weeks 1 to 18

# Create an empty list to store data
all_weeks_data = []

# Loop through each week and get the roster data
for week in weeks:
    week_data = []  # Temporary storage for current week's data

    for team in league.teams:
        team_name = team.team_name  # Fantasy Team Name

        for player in team.roster:
            week_data.append({
                "Team Name": team_name,
                "Week": week,
                "Player Name": player.name,
                "ESPN ID": player.playerId,  # ESPN Unique Player ID
                "Position": player.position if player.lineupSlot != "BE" else "Bench"
            })

    # If no data is returned for this week, stop iterating
    if not week_data:
        print(f"No data found for Week {week}. Stopping iteration.")
        break  # Stop looping when a week has no data

    # Add this week's data to the master list
    all_weeks_data.extend(week_data)

# Convert to DataFrame
df = pd.DataFrame(all_weeks_data)


            Team Name  Week      Player Name  ESPN ID Position
2875  bungalicious  💅    18       Gabe Davis  4243537    Bench
2876  bungalicious  💅    18       Jake Bates  4689936        K
2877  bungalicious  💅    18      Sam Darnold  3912547       QB
2878  bungalicious  💅    18       Colts D/ST   -16011     D/ST
2879  bungalicious  💅    18  Zach Charbonnet  4426385       RB


            Team Name  Week      Player Name  ESPN ID Position
0    Fightin' Furries     1      Tyreek Hill  3116406       WR
1    Fightin' Furries     1    Isiah Pacheco  4361529       RB
2    Fightin' Furries     1        Joe Mixon  3116385    Bench
3    Fightin' Furries     1  Patrick Mahomes  3139477       QB
4    Fightin' Furries     1   Dalton Kincaid  4385690    Bench
..                ...   ...              ...      ...      ...
155   bungalicious  💅     1       Gabe Davis  4243537    Bench
156   bungalicious  💅     1       Jake Bates  4689936        K
157   bungalicious  💅     1      Sam Darnold  3912547       QB
158   bungalicious  💅     1       Colts D/ST   -16011     D/ST
159   bungalicious  💅     1  Zach Charbonnet  4426385       RB

[160 rows x 5 columns]
